In [6]:
import io
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import json
import time
import threading
import random
import alpaca_trade_api as tradeapi
from openai import OpenAI

data_file = 'equities.json'

client = OpenAI(api_key="YOUR_OPENAI_API_KEY") # Replace with actual key
key = 'YOUR_ALPACA_API_KEY' # Replace with actual key
secret_key = 'YOUR_ALPACA_SECRET_KEY' # Replace with actual secretkey
base_url = 'https://paper-api.alpaca.markets/'

api = tradeapi.REST(key, secret_key, base_url, api_version="v2")

def fetch_open_orders():
    try:
        orders = api.list_orders(status='open')
        open_orders = []
        for order in orders:
            open_orders.append({
                'symbol': order.symbol,
                'qty': order.qty,
                'limit_price': order.limit_price,
                'side': 'buy',
                })
            return open_orders
    except Exception as e:
        print(f"Error fetching open orders: {e}")
        return [] 

def fetch_portfolio():
    try:
        positions = api.list_positions()
        portfolio = []
        for pos in positions:
            portfolio.append({
                'symbol': pos.symbol,
                'qty': pos.qty,
                'entry_price': pos.avg_entry_price,
                'unrealized_pl': pos.unrealized_pl,
                'current_price': pos.current_price,
                'side': 'long'
            })
        return portfolio
    except Exception as e:
        print(f"Error fetching portfolio: {e}")
        return []

def fetch_mock_api(symbol):
    return {
        "price" : 100
    }

def chatgpt_response(message):
    try:
        portfolio_data = fetch_portfolio()
        open_orders = fetch_open_orders()
        system_prompt = (
            "You are an AI Portfolio Manager. "
            "Your tasks are: "
            "1) Evaluate risk exposures of current holdings; "
            "2) Analyse open limit orders and their potential impact; "
            "3) Provide insights on portfolio health, diversification, and trade adjustments; "
            "4) Comment on market outlook based on general conditions (no real-time claims); "
            "5) Identify risks and suggest risk management strategies."
        )
        user_prompt = (
            f"Portfolio data (JSON-like): {portfolio_data}\n"
            f"Open orders (JSON-like): {open_orders}\n\n"
            f"User question: {message}"
        )
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.3,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error getting AI response: {e}"

class TradingBotGUI:
     
    def __init__(self, root):
        self.root = root
        self.root.title("AI Trading Bot")
        self.equities = self.load_equities()
        self.system_running = False

        self.form_frame = tk.Frame(root)
        self.form_frame.pack(pady=10)
          
        # Form to add a new equity to our trading bot
        tk.Label(self.form_frame, text="Symbol:").grid(row=0, column=0, padx=5)
        self.symbol_entry = tk.Entry(self.form_frame)
        self.symbol_entry.grid(row=0, column=1, padx=5)

        tk.Label(self.form_frame, text="Levels:").grid(row=0, column=2, padx=5)
        self.levels_entry = tk.Entry(self.form_frame)
        self.levels_entry.grid(row=0,column=3, padx=5)

        tk.Label(self.form_frame, text="Drawdown%:").grid(row=0, column=4, padx=5)
        self.drawdown_entry = tk.Entry(self.form_frame)
        self.drawdown_entry.grid(row=0,column=5, padx=5)
        
        self.add_button = tk.Button(self.form_frame, text="Add Equity", command=self.add_equity)
        self.add_button.grid(row=0, column=6, padx=5)

        # Table to track the traded equities
        self.tree = ttk.Treeview(root, columns=("Symbol", "Position", "Entry Price", "Levels", "Status"), show='headings')
        for col in ["Symbol", "Position", "Entry Price", "Levels", "Status"]:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100)
        self.tree.pack(pady=10)

        # Buttons to control the bot
        self.toggle_selected_system_button = tk.Button(root, text="Toggle Selected System", command=self.toggle_selected_system)
        self.toggle_selected_system_button.pack(pady=5)

        self.remove_button = tk.Button(root, text="Remove Selected Equity", command=self.remove_equity)
        self.remove_button.pack(pady=5)

        # AI component
        self.chat_frame = tk.Frame(root)
        self.chat_frame.pack(pady=10)

        self.chat_input = tk.Entry(self.chat_frame, width=50)
        self.chat_input.grid(row=0, column=0, padx=5)

        self.send_button = tk.Button(self.chat_frame, text="Send", command=self.send_message)
        self.send_button.grid(row=0, column=1, padx=5)

        self.chat_output = tk.Text(root, height=10, width=60, state=tk.DISABLED)
        self.chat_output.pack(pady=10)

        # Load saved data
        self.refresh_table()

        # Auto-refreshing
        self.running = True
        self.auto_update_thread = threading.Thread(target=self.auto_update, daemon=True)
        self.auto_update_thread.start()

    def add_equity(self):
        symbol = self.symbol_entry.get().upper()
        levels = self.levels_entry.get()
        drawdown = self.drawdown_entry.get()    

        if not symbol or not levels.isdigit() or not drawdown.replace('.','',1).isdigit():
                messagebox.showerror("Error", "Invalid Input")
                return
            
        levels = int(levels)
        drawdown = float(drawdown)/100
        entry_price = fetch_mock_api(symbol)['price']

        level_prices = {i+1: round(entry_price * (1-drawdown*(i+1)), 2) for i in range(levels)}

        self.equities[symbol] = {
                "position": 0,
                "entry_price": entry_price,
                "levels": level_prices,
                "drawdown": drawdown,
                "status": "Off"
            }
        self.save_equities()
        self.refresh_table()

    def toggle_selected_system(self):
        selected_items = self.tree.selection()
        if not selected_items:
                messagebox.showerror("Error", "No Equity Selected")
                return
        for item in selected_items:
                symbol = self.tree.item(item)['values'][0]
                self.equities[symbol]['status'] = "On" if self.equities[symbol]['status'] == "Off" else "Off"
            
        self.save_equities()
        self.refresh_table()

    def remove_equity(self):
        selected_items = self.tree.selection()
        if not selected_items:
            messagebox.showerror("Error", "No Equity Selected")
            return
        for item in selected_items:
            symbol = self.tree.item(item)['values'][0]
            if symbol in self.equities:
                del self.equities[symbol]
            
        self.save_equities()
        self.refresh_table()
        
    def send_message(self):
        message = self.chat_input.get()
        if not message:
            return
            
        response = chatgpt_response(message)

        self.chat_output.config(state=tk.NORMAL)
        self.chat_output.insert(tk.END, f"You: {message}\n{response}\n\n")
        self.chat_output.config(state=tk.DISABLED)
        self.chat_input.delete(0, tk.END)
    
    def fetch_market_data(self, symbol):
         try:
              barset = api.get_barset(symbol, 'day', limit=5)
              return {"price": barset.price}
         except Exception as e:
              return{"price":-1}
         
    def check_existing_positions(self, symbol, price):
        try:
            orders = api.list_orders(status="open", symbols=symbol)
            for order in orders:
                 if float(order.limit_price) == price:
                      return True
        except Exception as e:
             messagebox.showerror("API Error", f"Error Checking Orders {e}")
        return False
    
    def get_max_entry_price(self, symbol):
         try:
              orders = api.list_orders(status="filled", limit=50)
              prices = [float(order.limit_price) for order in orders if order.filled_avg_price and order.symbol == symbol]
              return max(prices) if prices else -1
         except Exception as e:
              messagebox.showerror("API Error", f"Error Getting Max Entry Price {e}")
              return 0
    
    def trade_systems(self):
         for symbol, data in self.equities.items():
              if data['status'] == "On":
                   position_exists = False
                   try:
                        position = api.get_position(symbol)
                        entry_price = self.get_max_entry_price(symbol)
                        position_exists = True
                   except Exception as e:
                        api.submit.order(
                            symbol=symbol,
                            qty=1,
                            side='buy',
                            type='market',
                            time_in_force='gtc', 
                        )
                        messagebox.showinfo("Trade Executed", f"Bought 1 share of {symbol} at market price.")
                        time.sleep(2)
                        entry_price = self.get_max_entry_price(symbol)
                   print(entry_price)
                   level_prices = {i+1: round(entry_price * (1-data['drawdown']*(i+1)), 2) for i in range(len(data['levels']))}
                   existing_levels = self.equities.get[symbol]['levels']
                   for level, price in level_prices.items():
                        if level not in existing_levels and -level not in existing_levels:
                             existing_levels[level] = price
                   self.equities[symbol]['entry_price'] = entry_price
                   self.equities[symbol]['levels'] = existing_levels
                   self.equities[symbol]['position'] = position.qty if position_exists else 1
                   for level, prices in level_prices.items():
                        if level in self.equities[symbol]['levels']:
                            self.place_order(symbol, price, level)

                   self.save_equities()
                   self.refresh_table()
              else:
                   return
    def place_order(self, symbol, price, level):
         if -level in self.equities[symbol]['levels'] or '-1' in self.equities[symbol]['levels'].keys():
              return
         try:
            api.submit_order(
                     symbol=symbol,
                     qty=1,
                     side='sell',
                     type='limit',
                     time_in_force='gtc',
                     limit_price=price
                )
            messagebox.showinfo("Trade Executed", f"Placed sell order for 1 share of {symbol} at £{price}.")
            del self.equities[symbol]['levels'][level]
         except Exception as e:
            messagebox.showerror("API Error", f"Error Placing Order {e}")
                   
    def refresh_table(self):
        for row in self.tree.get_children():
            self.tree.delete(row)

        for symbol, data in self.equities.items():
            self.tree.insert("", tk.END, values=(
                symbol, 
                data['position'], 
                data['entry_price'], 
                data['status']
            ))
        
    def auto_update(self):
        while self.running:
            time.sleep(5)
            self.trade_systems()

    def save_equities(self):
        with open(data_file, 'w') as f:
            json.dump(self.equities, f)
        
    def load_equities(self):
        try:
            with open(data_file, 'r') as f:
                return json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            return {}
        
    def on_close(self):
        self.running = False
        self.save_equities()
        self.root.destroy()
    
if __name__ == "__main__":
        root = tk.Tk()
        app = TradingBotGUI(root)
        root.protocol("WM_DELETE_WINDOW", app.on_close)
        root.mainloop()

Exception in thread Thread-8 (auto_update):
Traceback (most recent call last):
  File "C:\Users\enric\AppData\Local\Temp\ipykernel_5404\1125671597.py", line 245, in trade_systems
    position = api.get_position(symbol)
  File "C:\Users\enric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\alpaca_trade_api\rest.py", line 508, in get_position
    resp = self.get('/positions/{}'.format(symbol))
  File "C:\Users\enric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\alpaca_trade_api\rest.py", line 254, in get
    return self._request('GET', path, data)
           ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^
  File "C:\Users\enric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\alpaca_trade_api\rest.py", line 222, in _request
    return self._one_request(method, url, opts, retry)